### Necessary Library and Sources

In [12]:
# Source the R script
library(rvest)
library(dplyr)
library(readr)
library(lubridate)
library(stringr)
library(purrr)
library(TTR)  
source("data_updater.R")

### Updating Time series

In [2]:
# Run the function for a specific date
last_day <- "2025-10-22"
scrap_data_updater(last_day)

# Optional: replace zeros in your folder
replace_zero_with_close("./Scrapped_data/daily")


Done for 1JANATAMF 
Done for 1STPRIMFMF 
Done for AAMRANET 
Done for AAMRATECH 
Done for ABB1STMF 
Done for ABBANK 
Ticker file not found for ABBLPBOND 
Done for ACFL 
Done for ACI 
Done for ACIFORMULA 
Done for ACMELAB 
Done for ACMEPL 
Done for ACTIVEFINE 
Done for ADNTEL 
Done for ADVENT 
Done for AFCAGRO 
Done for AFTABAUTO 
Done for AGNISYSL 
Done for AGRANINS 
Done for AIBL1STIMF 
Ticker file not found for AIBLPBOND 
Done for AIL 
Done for AL-HAJTEX 
Done for ALARABANK 
Done for ALIF 
Done for ALLTEX 
Done for AMANFEED 
Done for AMBEEPHA 
Done for AMCL(PRAN) 
Done for ANLIMAYARN 
Done for ANWARGALV 
Done for AOL 
Done for APEXFOODS 
Done for APEXFOOT 
Done for APEXSPINN 
Done for APEXTANRY 
Done for APOLOISPAT 
Ticker file not found for APSCLBOND 
Done for ARAMIT 
Done for ARAMITCEM 
Done for ARGONDENIM 
Done for ASIAINS 
Done for ASIAPACINS 
Done for ASIATICLAB 
Done for ATCSLGF 
Done for ATLASBANG 
Done for AZIZPIPES 
Done for BANGAS 
Ticker file not found for BANKASI1PB 
Done 

Warning message:
"All formats failed to parse. No formats found."
Warning message:
"All formats failed to parse. No formats found."
Warning message:
"All formats failed to parse. No formats found."


Updated: 00DS30.csv 
Updated: 00DSES.csv 
Updated: 00DSEX.csv 
Updated: 1JANATAMF.csv 
Updated: 1STPRIMFMF.csv 
Updated: AAMRANET.csv 
Updated: AAMRATECH.csv 
Updated: ABB1STMF.csv 
Updated: ABBANK.csv 
Updated: ACFL.csv 
Updated: ACI.csv 
Updated: ACIFORMULA.csv 
Updated: ACMELAB.csv 
Updated: ACMEPL.csv 
Updated: ACTIVEFINE.csv 
Updated: ADNTEL.csv 
Updated: ADVENT.csv 
Updated: AFCAGRO.csv 
Updated: AFTABAUTO.csv 
Updated: AGNISYSL.csv 
Updated: AGRANINS.csv 
Updated: AIBL1STIMF.csv 
Updated: AIL.csv 
Updated: AL-HAJTEX.csv 
Updated: ALARABANK.csv 
Updated: ALIF.csv 
Updated: ALLTEX.csv 
Updated: AMANFEED.csv 
Updated: AMBEEPHA.csv 
Updated: AMCL(PRAN).csv 
Updated: ANLIMAYARN.csv 
Updated: ANWARGALV.csv 
Updated: AOL.csv 
Updated: APEXFOODS.csv 
Updated: APEXFOOT.csv 
Updated: APEXSPINN.csv 
Updated: APEXTANRY.csv 
Updated: APOLOISPAT.csv 
Updated: ARAMIT.csv 
Updated: ARAMITCEM.csv 
Updated: ARGONDENIM.csv 
Updated: ASIAINS.csv 
Updated: ASIAPACINS.csv 
Updated: ASIATICLAB.csv 
Up

### Generating Trade file

In [6]:
url <- paste0("https://www.dsebd.org/day_end_archive.php?startDate=",
                last_day, "&endDate=", last_day, "&inst=All%20Instrument&archive=data")
  
# Read all tables
tables <- read_html(url) %>% html_table(fill = TRUE)

# Pick the relevant table (like df_list[-2] in Python)
ohlcv_table <- tables[[length(tables)-1]]
# Rename columns to match your format
ohlcv_table <- ohlcv_table %>%
rename(
    Scrip = `TRADING CODE`,
    High = HIGH,
    Low = LOW,
    Close = `CLOSEP*`,
    Open = `OPENP*`,
    Volume = VOLUME,
    Trade = TRADE
) %>%
mutate(Date = last_day)%>%
mutate(
      Open = as.numeric(str_replace_all(Open, ",", "")),
      High = as.numeric(str_replace_all(High, ",", "")),
      Low = as.numeric(str_replace_all(Low, ",", "")),
      Close = as.numeric(str_replace_all(Close, ",", "")),
      Volume = as.numeric(str_replace_all(Volume, ",", "")),
      Trade = as.numeric(str_replace_all(Trade, ",", ""))
    )

### Adding Analytics

-> Checking 250 Days Breakout

In [10]:
# Create a list of tickers
tickers <- trimws(ohlcv_table$Scrip)  # remove any trailing spaces

# Function to check 250-day breakout
check_breakout <- function(ticker) {
  file_path <- paste0("Scrapped_data/daily/", ticker, ".csv")
  
  if (!file.exists(file_path)) {
    return(FALSE)  # CSV doesn't exist
  }
  
  # Read the CSV
  df <- read_csv(file_path, show_col_types = FALSE)
  
  # Make sure Close column is numeric
  df <- df %>%
    mutate(close = as.numeric(gsub(",", "", close)))
  
  if (nrow(df) < 250) {
    return(FALSE)  # Not enough data
  }
  
  # Get last 250 days
  last_250 <- tail(df$close, 250)
  
  # Check if the latest close is higher than the previous 249 highs
  latest_close <- last_250[250]
  max_prev <- max(last_250[1:249], na.rm = TRUE)
  
  return(latest_close > max_prev)
}

# Apply the function to all tickers
ohlcv_table <- ohlcv_table %>%
  mutate(Breakout = map_lgl(Scrip, check_breakout))
ohlcv_table

#,DATE,Scrip,LTP*,High,Low,Open,Close,YCP,Trade,VALUE (mn),Volume,Date,Breakout
<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>
1,2025-10-22,1JANATAMF,2.9,2.9,2.7,2.9,2.8,2.8,145,3.355,1201483,2025-10-22,FALSE
2,2025-10-22,1STPRIMFMF,20.1,20.1,19.1,19.6,19.9,19.6,373,6.949,354575,2025-10-22,FALSE
3,2025-10-22,AAMRANET,19.8,20.5,19.1,19.7,19.8,19.7,154,1.185,59798,2025-10-22,FALSE
4,2025-10-22,AAMRATECH,12.3,12.6,11.9,12.6,12.3,12.4,27,0.129,10506,2025-10-22,FALSE
5,2025-10-22,ABB1STMF,3,3.1,2.9,3.0,2.9,3,61,1.177,394726,2025-10-22,FALSE
6,2025-10-22,ABBANK,5.1,5.3,5.0,5.3,5.1,5.3,431,5.831,1131491,2025-10-22,FALSE
7,2025-10-22,ABBLPBOND,868,868.0,868.0,868.0,868.0,807.5,1,0.009,10,2025-10-22,FALSE
8,2025-10-22,ACFL,18.2,18.2,16.5,16.5,18.2,18.2,50,1.008,55576,2025-10-22,FALSE
9,2025-10-22,ACI,169.3,172.7,168.0,168.2,169.3,170.4,141,1.828,10816,2025-10-22,FALSE


-> Checking Bearish and Bullish Trend in the tickers

In [16]:
# Function to check bullish trend
check_bullish <- function(ticker) {
  file_path <- paste0("Scrapped_data/daily/", ticker, ".csv")
  
  if (!file.exists(file_path)) return(FALSE)
  
  df <- read_csv(file_path, show_col_types = FALSE)
  
  # Make sure close column exists and is numeric
  if (!"close" %in% colnames(df)) return(FALSE)
  df <- df %>% mutate(close = as.numeric(gsub(",", "", close)))
  
  if (nrow(df) < 10 + 5) return(FALSE)  # Need enough data for EMA/SMA
  
  ema10 <- EMA(df$close, n = 10)
  sma10 <- SMA(df$close, n = 10)
  
  # Compare EMA vs SMA for the last 5 days
  last_7 <- tail(seq_along(ema10), 10)
  
  all(ema10[last_7] > sma10[last_7], na.rm = TRUE)
}

# Function to check bearish trend
check_bearish <- function(ticker) {
  file_path <- paste0("Scrapped_data/daily/", ticker, ".csv")
  
  if (!file.exists(file_path)) return(FALSE)
  
  df <- read_csv(file_path, show_col_types = FALSE)
  
  if (!"close" %in% colnames(df)) return(FALSE)
  df <- df %>% mutate(close = as.numeric(gsub(",", "", close)))
  
  if (nrow(df) < 10 + 5) return(FALSE)
  
  ema10 <- EMA(df$close, n = 10)
  sma10 <- SMA(df$close, n = 10)
  
  last_7 <- tail(seq_along(ema10), 7)
  
  all(ema10[last_7] < sma10[last_7], na.rm = TRUE)
}

# Apply to your ohlcv_table
ohlcv_table <- ohlcv_table %>%
  mutate(
    Bullish = map_lgl(Scrip, check_bullish),
    Bearish = map_lgl(Scrip, check_bearish)
  )

ohlcv_table

#,DATE,Scrip,LTP*,High,Low,Open,Close,YCP,Trade,VALUE (mn),Volume,Date,Breakout,Bullish,Bearish
<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<lgl>,<lgl>
1,2025-10-22,1JANATAMF,2.9,2.9,2.7,2.9,2.8,2.8,145,3.355,1201483,2025-10-22,FALSE,FALSE,TRUE
2,2025-10-22,1STPRIMFMF,20.1,20.1,19.1,19.6,19.9,19.6,373,6.949,354575,2025-10-22,FALSE,FALSE,FALSE
3,2025-10-22,AAMRANET,19.8,20.5,19.1,19.7,19.8,19.7,154,1.185,59798,2025-10-22,FALSE,FALSE,FALSE
4,2025-10-22,AAMRATECH,12.3,12.6,11.9,12.6,12.3,12.4,27,0.129,10506,2025-10-22,FALSE,FALSE,FALSE
5,2025-10-22,ABB1STMF,3,3.1,2.9,3.0,2.9,3,61,1.177,394726,2025-10-22,FALSE,FALSE,FALSE
6,2025-10-22,ABBANK,5.1,5.3,5.0,5.3,5.1,5.3,431,5.831,1131491,2025-10-22,FALSE,FALSE,TRUE
7,2025-10-22,ABBLPBOND,868,868.0,868.0,868.0,868.0,807.5,1,0.009,10,2025-10-22,FALSE,FALSE,FALSE
8,2025-10-22,ACFL,18.2,18.2,16.5,16.5,18.2,18.2,50,1.008,55576,2025-10-22,FALSE,FALSE,FALSE
9,2025-10-22,ACI,169.3,172.7,168.0,168.2,169.3,170.4,141,1.828,10816,2025-10-22,FALSE,FALSE,FALSE


-> Generating Monthly Return

In [17]:
# Function to calculate 20-day return
calc_20day_return <- function(ticker) {
  file_path <- paste0("Scrapped_data/daily/", ticker, ".csv")
  
  if (!file.exists(file_path)) return(NA_real_)
  
  df <- read_csv(file_path, show_col_types = FALSE)
  
  if (!"close" %in% colnames(df)) return(NA_real_)
  
  df <- df %>% mutate(close = as.numeric(gsub(",", "", close)))
  
  if (nrow(df) < 20) return(NA_real_)
  
  last_20 <- tail(df$close, 20)
  
  # Calculate return: (latest / earliest) - 1
  ret <- (last_20[20] / last_20[1]) - 1
  
  return(ret)
}

# Apply to ohlcv_table
ohlcv_table <- ohlcv_table %>%
  mutate(MonthlyReturn = map_dbl(Scrip, calc_20day_return))

ohlcv_table

#,DATE,Scrip,LTP*,High,Low,Open,Close,YCP,Trade,VALUE (mn),Volume,Date,Breakout,Bullish,Bearish,MonthlyReturn
<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<lgl>,<lgl>,<dbl>
1,2025-10-22,1JANATAMF,2.9,2.9,2.7,2.9,2.8,2.8,145,3.355,1201483,2025-10-22,FALSE,FALSE,TRUE,-0.06666667
2,2025-10-22,1STPRIMFMF,20.1,20.1,19.1,19.6,19.9,19.6,373,6.949,354575,2025-10-22,FALSE,FALSE,FALSE,-0.11946903
3,2025-10-22,AAMRANET,19.8,20.5,19.1,19.7,19.8,19.7,154,1.185,59798,2025-10-22,FALSE,FALSE,FALSE,-0.06161137
4,2025-10-22,AAMRATECH,12.3,12.6,11.9,12.6,12.3,12.4,27,0.129,10506,2025-10-22,FALSE,FALSE,FALSE,-0.13380282
5,2025-10-22,ABB1STMF,3,3.1,2.9,3.0,2.9,3,61,1.177,394726,2025-10-22,FALSE,FALSE,FALSE,-0.06451613
6,2025-10-22,ABBANK,5.1,5.3,5.0,5.3,5.1,5.3,431,5.831,1131491,2025-10-22,FALSE,FALSE,TRUE,0.02000000
7,2025-10-22,ABBLPBOND,868,868.0,868.0,868.0,868.0,807.5,1,0.009,10,2025-10-22,FALSE,FALSE,FALSE,NA
8,2025-10-22,ACFL,18.2,18.2,16.5,16.5,18.2,18.2,50,1.008,55576,2025-10-22,FALSE,FALSE,FALSE,-0.04712042
9,2025-10-22,ACI,169.3,172.7,168.0,168.2,169.3,170.4,141,1.828,10816,2025-10-22,FALSE,FALSE,FALSE,-0.05260213


-> Generating Daily Return

In [18]:
# Function to calculate daily return
calc_daily_return <- function(ticker) {
  file_path <- paste0("Scrapped_data/daily/", ticker, ".csv")
  
  if (!file.exists(file_path)) return(NA_real_)
  
  df <- read_csv(file_path, show_col_types = FALSE)
  
  if (!"close" %in% colnames(df)) return(NA_real_)
  
  df <- df %>% mutate(close = as.numeric(gsub(",", "", close)))
  
  if (nrow(df) < 2) return(NA_real_)
  
  # Daily return: (latest / previous day) - 1
  ret <- (df$close[nrow(df)] / df$close[nrow(df)-1]) - 1
  
  return(ret)
}

# Apply to ohlcv_table
ohlcv_table <- ohlcv_table %>%
  mutate(DailyReturn = map_dbl(Scrip, calc_daily_return))

ohlcv_table


#,DATE,Scrip,LTP*,High,Low,Open,Close,YCP,Trade,VALUE (mn),Volume,Date,Breakout,Bullish,Bearish,MonthlyReturn,DailyReturn
<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>
1,2025-10-22,1JANATAMF,2.9,2.9,2.7,2.9,2.8,2.8,145,3.355,1201483,2025-10-22,FALSE,FALSE,TRUE,-0.06666667,0.000000000
2,2025-10-22,1STPRIMFMF,20.1,20.1,19.1,19.6,19.9,19.6,373,6.949,354575,2025-10-22,FALSE,FALSE,FALSE,-0.11946903,0.015306122
3,2025-10-22,AAMRANET,19.8,20.5,19.1,19.7,19.8,19.7,154,1.185,59798,2025-10-22,FALSE,FALSE,FALSE,-0.06161137,0.005076142
4,2025-10-22,AAMRATECH,12.3,12.6,11.9,12.6,12.3,12.4,27,0.129,10506,2025-10-22,FALSE,FALSE,FALSE,-0.13380282,-0.008064516
5,2025-10-22,ABB1STMF,3,3.1,2.9,3.0,2.9,3,61,1.177,394726,2025-10-22,FALSE,FALSE,FALSE,-0.06451613,-0.033333333
6,2025-10-22,ABBANK,5.1,5.3,5.0,5.3,5.1,5.3,431,5.831,1131491,2025-10-22,FALSE,FALSE,TRUE,0.02000000,-0.037735849
7,2025-10-22,ABBLPBOND,868,868.0,868.0,868.0,868.0,807.5,1,0.009,10,2025-10-22,FALSE,FALSE,FALSE,NA,NA
8,2025-10-22,ACFL,18.2,18.2,16.5,16.5,18.2,18.2,50,1.008,55576,2025-10-22,FALSE,FALSE,FALSE,-0.04712042,0.000000000
9,2025-10-22,ACI,169.3,172.7,168.0,168.2,169.3,170.4,141,1.828,10816,2025-10-22,FALSE,FALSE,FALSE,-0.05260213,-0.006455399


### Final Csv File

In [19]:
write_csv(ohlcv_table, "trade_data.csv")